# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#csv_file = os.path.join('..', 'WeatherPy', 'output_data', 'city_weather.csv')
csv_file = os.path.join('..','output_data', 'cities.csv')
weathercity_df = pd.read_csv(csv_file)
weathercity_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations = weathercity_df[["Lat", "Lng"]]
humidity = weathercity_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 4


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

weathercity_df = weathercity_df[(weathercity_df['Max Temp'] >= 70) & 
                                  (weathercity_df['Max Temp'] <= 87) &
                                  (weathercity_df['Cloudiness'] <= 3) &
                                  (weathercity_df['Wind Speed'] <= 6)] 

weathercity_df.dropna
 

<bound method DataFrame.dropna of      City_ID             City  Cloudiness Country        Date  Humidity  \
7          7            opuwo           0     NaN  1558378523        23   
70        70     ponta do sol           0      BR  1558378767        57   
87        87         ambilobe           2      MG  1558378770        92   
88        88         cidreira           0      BR  1558378770        66   
125      125  arraial do cabo           0      BR  1558378491        78   
139      139            vaini           0      IN  1558378780        88   
142      142        sarai mir           0      IN  1558378781        35   
297      297            mogok           0      MM  1558378844        51   
494      494            baylo           1      PH  1558378889        69   

       Lat     Lng  Max Temp  Wind Speed  
7   -18.06   13.84     72.06        5.44  
70  -20.63  -46.00     73.68        5.32  
87  -13.19   49.05     74.76        4.52  
88  -30.17  -50.22     76.56        5.50  


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:

#hotel_df = weathercity_df.loc[:,['Lat','Lng','City','Country']]
weathercity_df=weathercity_df.rename(columns={'Lat':'Latitude','Lng':'Longitude'})
weathercity_df.dropna()

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07


In [ ]:
#find hotel infoformation
hotelname = []
hoteladdress =[]
hotelrating =[]
hotellat =[]
hotellng =[]

for index, row in hotel_df.iterrows():
    #current info
    currentCity = row["City"]
    currentCountry = row["Country"]
    
    #geocoordinates
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_radius = 5000 #this is in meters
    target_type = "hotel"
    target_keyword = "hotel"

    #set parameters
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run request using params dictionary
    response = requests.get(base_url,params=params).json()
    #simplify finding results
    results = response['results']
     
#     try:
#         hotelName.append(results[0]['name'])
#         hotelAddress.append(results[0]['vicinity'])
#         hotelRating.append(results[0]['rating'])
#         hotelLat.append(results[0]['geometry']['location']['lat'])
#         hotelLng.append(results[0]['geometry']['location']['lng'])
#         print(f"Processeing Record for a {target_keyword} in {currentCity}, {currentCountry}")
#     except:
#         hotelName.append(np.nan)
#         hotelAddress.append(np.nan)
#         hotelRating.append(np.nan)
#         hotelLat.append(np.nan)
#         hotelLng.append(np.nan)
#         print(f"No results/{target_keyword} found in {currentCity}, {currentCountry}. Skipping...")  
  

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info.describe
#locations = hotel_df[["Lat", "Lng"]]
#locations

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure
